In [ ]:
%matplotlib inline

from __future__ import division
import numpy as np
import menpo.io as mio
from menpo.image import Image
from menpo.feature import ndfeature, greyscale, centralize
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from alaborticcv2015.deepconvkernel.generative import GenerativeLDCN, learn_pca_filters

In [ ]:
@ndfeature
def grey_igo(pixels):
    pixels = greyscale(pixels)
    pixels = igo (pixels)
    return pixels

def greyscale_average(pixels):
    return greyscale(pixels, mode='average')

# Generate PCA Net:

Load data:

In [ ]:
folder_path = '/data/PhD/alaborticcv2015/'

In [ ]:
images = []
for i in mio.import_images('/data/PhD/DataBases/faces/lfpw/**/*', verbose=True, 
                           max_images=1000):
    i.crop_to_landmarks_proportion_inplace(0.5)
    i = i.rescale_landmarks_to_diagonal_range(100)
    labeller(i, 'PTS', ibug_face_66)
    images.append(i)

In [ ]:
visualize_images(images)

In [ ]:
# general network options
features = greyscale_average
architecture = 1
normalize_patches = centralize
normalize_filters = None
landmarks = True
stride = (4, 4)

# feature images
images = [features(i) for i in images] 

# number of filters and patch sizes
n_filters = range(1, 16, 1)
patch_shape = [ps for ps in range(2, 11, 1)]

In [ ]:
for nf in n_filters:
    for ps in patch_shape:
        
        if nf < ps * ps:

            net = GenerativeLDCN(learn_filters=learn_pca_filters, 
                                 n_filters=np.minimum(nf, ps * ps -1),
                                 n_layers=1,
                                 architecture=architecture,
                                 normalize_patches=normalize_patches,
                                 normalize_filters=normalize_filters,
                                 patch_shape=(ps, ps))
            if landmarks:
                net.learn_network_from_landmarks(images, group='ibug_face_66')
            else:
                net.learn_network_from_grid(images, stride=stride)

            file_name = (folder_path
                         + 'networks/pca_net/pca_net_'
                         + features.__name__ + '_'
                         + str(1) + '_' 
                         + str(architecture) + '_'
                         + str(nf) + '_' 
                         + str(ps) + '_'
                         + str(landmarks) + '.pkl.gz') 
            mio.export_pickle(net, file_name, overwrite=True)

            print '- network {}, {}: '.format(nf, ps)

## PCA Net2:

In [ ]:
for nf1 in n_filters:
    for nf2 in n_filters:
        for ps in patch_shape:

            if nf1 < ps * ps or nf2 < ps * ps:

                net = GenerativeLDCN(learn_filters=learn_pca_filters, 
                                     n_filters=[nf1, nf2],
                                     n_layers=2,
                                     architecture=architecture,
                                     normalize_patches=normalize_patches,
                                     normalize_filters=normalize_filters,
                                     patch_shape=(ps, ps))
                if landmarks:
                    net.learn_network_from_landmarks(images, group='ibug_face_66')
                else:
                    net.learn_network_from_grid(images, stride=(4, 4))

                file_name = (folder_path
                             + 'networks/pca_net/pca_net_'
                             + features.__name__ + '_'
                             + str(2) + '_' 
                             + str(architecture) + '_'
                             + str(nf) + '_' 
                             + str(ps) + '_'
                             + str(landmarks) + '.pkl.gz') 
                mio.export_pickle(net, file_name, overwrite=True)

                print '- network {}, {}: '.format(nf, ps)